In [2]:
import os
import re

In [3]:
def list_wiki(dirname) :
    filepaths = []
    filenames = os.listdir(dirname)
    
    for filename in filenames :
        filepath = os.path.join(dirname, filename)
        
        if os.path.isdir(filepath) :
            filepaths.extend(list_wiki(filepath))
        else :
            find = re.findall(r"wiki_[0-9][0-9]",filepath)
            if 0 < len(find) :
                filepaths.append(filepath)
                
    return sorted(filepaths)

In [4]:
filepaths = list_wiki("F:/Study/DataSet/text")

In [5]:
print(filepaths[:5])
print(len(filepaths))

['F:/Study/DataSet/text\\AA\\wiki_00', 'F:/Study/DataSet/text\\AA\\wiki_01', 'F:/Study/DataSet/text\\AA\\wiki_02', 'F:/Study/DataSet/text\\AA\\wiki_03', 'F:/Study/DataSet/text\\AA\\wiki_04']
676


In [43]:
def combine_wiki(filepaths) :
    with open("DataSet/output_wiki_file.txt","w", encoding='utf-8') as outfile :
        for filename in filepaths :
            with open(filename,"r", encoding='utf-8') as infile :
                contents = infile.read()
                outfile.write(contents)

In [44]:
combine_wiki(filepaths)

In [52]:
f = open("DataSet/output_wiki_file.txt",'r', encoding="utf8")

for i in range(30) :
    line = f.readline()
    print(i," Line : ",line)
    
f.close()

0  Line :  <doc id="16" url="?curid=16" title="dictionary">

1  Line :  dictionary

2  Line :  

3  Line :  English.

4  Line :  Etymology.

5  Line :  , from , from , from , perfect past participle of + . .

6  Line :  

7  Line :  </doc>

8  Line :  <doc id="19" url="?curid=19" title="free">

9  Line :  free

10  Line :  

11  Line :  English.

12  Line :  Etymology.

13  Line :  From , , , from , from , from , from , from . Related to .

14  Line :  Germanic cognates include , , , , , and .

15  Line :  Other cognates include and .

16  Line :  Germanic and Celtic are the only Indo-European language branches in which the PIE word with the meaning of "dear, beloved" acquired the additional meaning of "free" in the sense of "not in bondage". This was an extension of the idea of "characteristic of those who are dear and beloved", in other words friends and tribe members (in contrast to unfree inhabitants from other tribes and prisoners of war, many of which were among the slaves – comp

In [71]:
#pre-processing
import re
from nltk.tokenize import sent_tokenize
import ray

In [72]:
with open("DataSet/output_wiki_file.txt",'r', encoding="utf8") as wiki_f :
    wiki = wiki_f.read()

In [73]:
ray.init()

2023-02-05 04:13:02,435	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.8.5
Ray version:,2.2.0


In [74]:
@ray.remote
def pre_processing1(wiki) :
    content_text = re.sub("<.*>", "", wiki, flags=re.MULTILINE)    
    #content_text = content_text.lower()
    content_text = re.sub(", from ", "", content_text, flags=re.MULTILINE)
    content_text = re.sub(" ,", "", content_text, flags=re.MULTILINE)
    content_text = re.sub("from .", "", content_text, flags=re.MULTILINE)
    content_text = re.sub(" \.", "", content_text, flags=re.MULTILINE)
    content_text = re.sub("\+", "", content_text, flags=re.MULTILINE)
    content_text = re.sub(" ;", "", content_text, flags=re.MULTILINE)
    
    return content_text

In [75]:
temp = pre_processing1.remote(wiki)
content_text = ray.get(temp)
print(content_text[:1000])

RayTaskError: [36mray::pre_processing1()[39m (pid=24340, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 789, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 814, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 506, in ray._raylet.raise_if_dependency_failed
ray.exceptions.RaySystemError: System error: 
traceback: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\_private\serialization.py", line 216, in _deserialize_msgpack_data
    obj = MessagePackSerializer.loads(msgpack_data, _python_deserializer)
  File "python\ray\includes/serialization.pxi", line 191, in ray._raylet.MessagePackSerializer.loads
  File "python\ray\includes/serialization.pxi", line 192, in ray._raylet.MessagePackSerializer.loads
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
MemoryError

During handling of the above exception, another exception occurred:

[36mray::pre_processing1()[39m (pid=24340, ip=127.0.0.1

In [ ]:
@ray.remote
def pre_processing2(content_text) :
    sent_tokenized = sent_tokenize(content_text)
    #sent_tokenized = [line.split("\n") for line in sent_tokenized]
    sent_tokenized = [re.sub("(\n)+", " ", line, flags=re.MULTILINE) for line in sent_tokenized]
    sent_tokenized = [re.sub('^[^a-zA-Z"\'(]', "", line, flags=re.MULTILINE) for line in sent_tokenized]
    
    return sent_tokenized

In [ ]:
temp = pre_processing2.remote(content_text[:20000])
sent_tokenized = ray.get(temp)

In [ ]:
for line in sent_tokenized :
    print(line)

In [ ]:
print(sent_tokenized)

In [ ]:
ray.shutdown()

(pre_processing1 pid=24340) 2023-02-05 04:13:10,295	ERROR serialization.py:371 -- 
(pre_processing1 pid=24340) Traceback (most recent call last):
(pre_processing1 pid=24340)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\_private\serialization.py", line 216, in _deserialize_msgpack_data
(pre_processing1 pid=24340)     obj = MessagePackSerializer.loads(msgpack_data, _python_deserializer)
(pre_processing1 pid=24340)   File "python\ray\includes/serialization.pxi", line 191, in ray._raylet.MessagePackSerializer.loads
(pre_processing1 pid=24340)   File "python\ray\includes/serialization.pxi", line 192, in ray._raylet.MessagePackSerializer.loads
(pre_processing1 pid=24340)   File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
(pre_processing1 pid=24340) MemoryError
(pre_processing1 pid=24340) 
(pre_processing1 pid=24340) During handling of the above exception, another exception occurred:
(pre_processing1 pid=24340) 
(pre_processing1 pid=24340) Traceback (most recent 